In [4]:
import pandas as pd
import dask.dataframe as dd

In [9]:
from ..Utils.cols_list import *

ImportError: attempted relative import with no known parent package

In [5]:
cols_list = [
    'NU_ANO_CENSO',
    'SG_UF',
    'CO_MUNICIPIO',
    'NO_MUNICIPIO',
    'CO_ENTIDADE',
    'TP_DEPENDENCIA',
    'TP_LOCALIZACAO',
    'TP_SITUACAO_FUNCIONAMENTO',
    'IN_AGUA_POTAVEL',
    'IN_ENERGIA_INEXISTENTE',
    'IN_ESGOTO_INEXISTENTE',
    'IN_AREA_VERDE',
    'IN_BANHEIRO',
    'IN_BIBLIOTECA',
    'IN_LABORATORIO_INFORMATICA',
    'IN_INTERNET',
    'IN_INTERNET_APRENDIZAGEM',
    'IN_INTERNET_ADMINISTRATIVO',
    'IN_INTERNET_COMUNIDADE',
    'IN_ACESSO_INTERNET_COMPUTADOR',
    'IN_ACES_INTERNET_DISP_PESSOAIS',
    'TP_REDE_LOCAL',
    'IN_BANDA_LARGA',
    'IN_EXAME_SELECAO',
    'QT_DOC_MED'
]

In [6]:
censo = dd.read_csv(
    '../../Dados/Censo Escolar/2021/dados/microdados_ed_basica_2021.csv', 
    sep=';', 
    encoding='latin-1',
    usecols=cols_list,
    assume_missing=True,
    )

In [7]:
censo.head()

,NU_ANO_CENSO,SG_UF,NO_MUNICIPIO,CO_MUNICIPIO,CO_ENTIDADE,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_SITUACAO_FUNCIONAMENTO,IN_AGUA_POTAVEL,IN_ENERGIA_INEXISTENTE,...,IN_INTERNET,IN_INTERNET_ADMINISTRATIVO,IN_INTERNET_APRENDIZAGEM,IN_INTERNET_COMUNIDADE,IN_ACESSO_INTERNET_COMPUTADOR,IN_ACES_INTERNET_DISP_PESSOAIS,TP_REDE_LOCAL,IN_BANDA_LARGA,IN_EXAME_SELECAO,QT_DOC_MED
0,2021.0,RO,Alta Floresta D'Oeste,1100015.0,11022558.0,2.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,NaN,0.0,0.0
1,2021.0,RO,Alta Floresta D'Oeste,1100015.0,11024275.0,2.0,1.0,1.0,1.0,0.0,...,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
2,2021.0,RO,Alta Floresta D'Oeste,1100015.0,11024291.0,3.0,2.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,NaN,0.0,0.0
3,2021.0,RO,Alta Floresta D'Oeste,1100015.0,11024372.0,3.0,1.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,2021.0,RO,Alta Floresta D'Oeste,1100015.0,11024666.0,3.0,2.0,1.0,1.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


### Inferindo medidas de conectividade / acesso à tecnologia nas escolas e agregando um score à nível de município para matching com outras bases de dados

Realizar agregações:
- Por município
- Por tipo de dependência
- Por tipo de localização
Medindo as variáveis avaliadas como explicativas e criando um score para cada nas agregações descritas.

In [8]:
recife=censo[censo['NO_MUNICIPIO']=='Recife'].head()

In [16]:
def get_agg(df, index_cols, agg_funs):
    '''
    Aggregates dataframe columns by a given list of agg functions.
    
    Parameters
    ----------
    df : pandas.DataFrame
        Dataframe to be aggregated.
    index_cols : list
        List of columns to be aggregated.
    agg_funs : dict
        dict of aggregation functions to be applied in each key pair.
    '''
    df=df.groupby(index_cols).agg(agg_funs)
    df.columns = ['_'.join(col) for col in df.columns.values]
    df.reset_index(inplace=True)
    return df


In [12]:
index_cols = ['NU_ANO_CENSO', 'SG_UF', 'CO_MUNICIPIO', 'TP_DEPENDENCIA', 'TP_LOCALIZACAO', 'TP_SITUACAO_FUNCIONAMENTO']
agg_funs = ['count', 'mean']
agg_recife = get_agg(recife, index_cols, agg_funs)

/tmp/ipykernel_743/1440307302.py:14: FutureWarning: ['NO_MUNICIPIO'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  df=df.groupby(index_cols).agg(agg_funs)


In [15]:
agg_recife

,NU_ANO_CENSO,SG_UF,CO_MUNICIPIO,TP_DEPENDENCIA,TP_LOCALIZACAO,TP_SITUACAO_FUNCIONAMENTO,CO_ENTIDADE_count,CO_ENTIDADE_mean,IN_AGUA_POTAVEL_count,IN_AGUA_POTAVEL_mean,...,IN_ACES_INTERNET_DISP_PESSOAIS_count,IN_ACES_INTERNET_DISP_PESSOAIS_mean,TP_REDE_LOCAL_count,TP_REDE_LOCAL_mean,IN_BANDA_LARGA_count,IN_BANDA_LARGA_mean,IN_EXAME_SELECAO_count,IN_EXAME_SELECAO_mean,QT_DOC_MED_count,QT_DOC_MED_mean
0,2021.0,PE,2611606.0,3.0,1.0,1.0,1,26117576.0,1,1.0,...,1,1.00,1,3.0,1,1.0,1,0.0,1,0.0
1,2021.0,PE,2611606.0,4.0,1.0,1.0,4,26115825.5,4,1.0,...,4,0.25,4,4.5,4,1.0,4,0.0,4,0.0
